# LAB 2: Genetic Algorithms. 
### Pygmo Optimization for Himmelblau, Rastrigin and Rosenbrock Functions
### AI and Machine Learning // Suchkova Natalia М8О-114М-22
30.10.2022 @ MAI IT-Center

## Table of Contents

1. [**Self-adaptive Differential Evolution (sade)**](#gen1)



2. [**Grey Wolf Optimizer (gwo)**](#gen2)



3. [**Extended Ant Colony Optimization algorithm (gaco)**](#gen3)


4. [**Own Defined Optimization Problem**](#gen4)



In [297]:
import pygmo as pg
from pygmo import *
from IPython.display import Image
from IPython.core.display import HTML
import pandas as pd

In [298]:
pg.__version__

'2.13.0'

In [447]:
df = pd.DataFrame(columns=['Problem', 'Method', 'Pop_Size',
                           'Num_Geners', 'Num_Evoluts', 
                           'Min_Coordinates', 'Function_Value'])

<a id='gen1'></a>

## Self-adaptive Differential Evolution (SADE)

### Himmelblau

**Функция Химмельблау**	$f(x,y) = (x^2 + y - 11)^2 + (x + y^2 - 7)^2 $

Напомним, как должен выглядеть *оптимумальный\ые минимум\ы*:

In [448]:
Image(url= "https://wikimedia.org/api/rest_v1/media/math/render/svg/3b6d888196974f862467cfe02748f3ae2ffdf476")

In [449]:
class HimmelblauOptimization: 
    """
    Задаем проблему Химмельблау, так как не нашла ее в pygmo почему-то :/
    This class defines a PyGMO-compatible User-Defined Optimization Problem.
    Конструктор  класса
    """

    def __init__(self,
                 x_min: float,
                 x_max: float,
                 y_min: float,
                 y_max: float): 
        
        # задаем границы определения функции

        self.x_min = x_min
        self.x_max = x_max
        self.y_min = y_min
        self.y_max = y_max

    def get_bounds(self):
        return ([self.x_min, self.y_min], [self.x_max, self.y_max])

    def fitness(self, x: list): # вычисляем значение функции
        
        function_value = (x[0] * x[0] + x[1] - 11.0) ** 2 + (x[0] + x[1] * x[1] - 7.0) ** 2
        return [function_value]

In [450]:
print(problem(HimmelblauOptimization(-5.12, 5.12, -5.12, 5.12)))

Problem name: <class '__main__.HimmelblauOptimization'>
	Global dimension:			2
	Integer dimension:			0
	Fitness dimension:			1
	Number of objectives:			1
	Equality constraints dimension:		0
	Inequality constraints dimension:	0
	Lower bounds: [-5.12, -5.12]
	Upper bounds: [5.12, 5.12]
	Has batch fitness evaluation: false

	Has gradient: false
	User implemented gradient sparsity: false
	Has hessians: false
	User implemented hessians sparsity: false

	Fitness evaluations: 0

	Thread safety: none



In [451]:
problem_Him = problem(HimmelblauOptimization(-5.12, 5.12, -5.12, 5.12))

In [452]:
pop_size = 100  # зададим внешние переменные ака характеристики популяции,
current_seed = 23 # чтобы условия у всех адгоритмов были одинаковые
number_of_evolutions = 10

In [510]:
def few_evols(algo, pop, gens=10,
              number_of_evols=10):
    
    """
    algo (pygmo obj) - алгоритм для оптимизации
    pop (pygmo obj) - популяция
    number_of_evols (int) - количество раз эволюции
    """
    if algo is pg.gaco:
        cur_algo = algorithm(algo(10, 13, 1.0, 1e9, 0.0, 1, 7, 100000, 100000, 0.0, False, 23))
        print('я бля работаю!!!!')
    else:
        cur_algo = algorithm(algo(gen=gens)) # задаем параметры алгоритма
    cur_algo.set_verbosity(2)  # частота лагирования
    individuals_list = []
    fitness_list = []
    
    # Заставляем популяцию эволюционировать
    for i in range(number_of_evols):
        pop = cur_algo.evolve(pop)  # запускаем эволюцию  начальной популяции
        individuals_list.append(pop.get_x()[pop.best_idx()])
        fitness_list.append(pop.get_f()[pop.best_idx()])
    
    log = cur_algo.extract(algo).get_log() # можно высети лог (историю лагирования) каждые n итераций из set_verbosity()
    
    # колдуем над датафреймом
    method = str(pop)
    if 'class' in method:
        method = method[31:41]
    elif 'Rastrigin' in method:
        method = method[14:23]
    else:
        method = method[31:41]
    data = [method, str(algo)[8:-2], pop_size, gens, 
            number_of_evols, pop.champion_x, pop.champion_f]
    df.loc[len(df)] = data
    
    return pop, individuals_list, fitness_list, log, df

    '''
    1/ добавить норм вывод
    2. добавить визуализацию

    3. доавить датафрейм с результатами
    # Вытащим лучших индивидов из всех поколений
    best_x = [ind[0] for ind in individuals_list]
    best_y = [ind[1] for ind in individuals_list]

    # Выразим границы задачи
    (x_min, y_min), (x_max, y_max) = udp.get_bounds()

    # Изобразим минимизацию за все поколения
    fig, ax = plt.subplots(figsize=(9, 5))
    ax.plot(np.arange(0, number_of_evolutions), fitness_list, label='Значение функции')

    # Выведем лучший образец
    champion_n = np.argmin(np.array(fitness_list))
    ax.scatter(champion_n, np.min(fitness_list), marker='x', color='r', label='Чемпион за всё время')

    # Оформление
    ax.set_xlim((0, number_of_evolutions))
    ax.grid('major')
    ax.set_title('Лучший представитель для каждого поколения', fontweight='bold')
    ax.set_xlabel('Номер поколения')
    ax.set_ylabel(r'Значение функции Химмельблау $f(x,y)$')
    ax.legend(loc='upper right')
    ax.set_yscale('log')
    plt.tight_layout()

    # Показ
    plt.show()


    # Изобразим функцию Химмельблау
    grid_points = 100
    x_vector = np.linspace(x_min, x_max, grid_points)
    y_vector = np.linspace(y_min, y_max, grid_points)
    x_grid, y_grid = np.meshgrid(x_vector, y_vector)
    z_grid = np.zeros((grid_points, grid_points))
    for i in range(x_grid.shape[1]):
        for j in range(x_grid.shape[0]):
            z_grid[i, j] = udp.fitness([x_grid[i, j], y_grid[i, j]])[0]

    # Create figure
    fig, ax = plt.subplots(figsize=(9,5))
    cs = ax.contour(x_grid, y_grid, z_grid, 50)

    # Показываем лучших представителей за каждое поколение
    ax.scatter(best_x, best_y, marker='x', color='r')

    # Оформление
    ax.set_xlim((x_min, x_max))
    ax.set_ylim((y_min, y_max))
    ax.set_title('Функция Химмельблау', fontweight='bold')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    cbar = fig.colorbar(cs)
    cbar.ax.set_ylabel(r'Значение функции Химмельблау $f(x,y)$')
    plt.tight_layout()

    # Показ
    plt.show()
    '''

In [454]:
# можно еще посмотреть информацию обалгоритмах
algo1 = algorithm(sade(gen=10))
print(algo1)

Algorithm name: saDE: Self-adaptive Differential Evolution [stochastic]
	Thread safety: basic

Extra info:
	Generations: 10
	Variant: 2
	Self adaptation variant: 1
	Stopping xtol: 1e-06
	Stopping ftol: 1e-06
	Memory: false
	Verbosity: 0
	Seed: 1452041765


In [456]:
pop_him = population(problem_Him, size=pop_size, seed=current_seed)

pop_him = few_evols(sade, pop_him)

pop_him[1]  # individuals_list 
pop_him[2]  # fitness_list

[array([0.00956613]),
 array([0.00956613]),
 array([0.00136954]),
 array([0.00010006]),
 array([7.52380033e-05]),
 array([3.51592259e-07]),
 array([4.01355661e-10]),
 array([2.85897783e-10]),
 array([2.85897783e-10]),
 array([2.85897783e-10])]

In [457]:
pop_him[0].champion_x

array([3.00000302, 1.9999979 ])

In [458]:
pop_him[0].champion_f

array([2.85897783e-10])

In [459]:
pop_him[3] # выводим лог

[]

### Rastrigin

**Функция Растригина** $f(x) = An + \displaystyle\sum_{i=1}^{n} \big\lbrack  x_i^2 - Acos(2pix_i) \big\rbrack$, where $A = 10$

In [460]:
Image(url= "https://wikimedia.org/api/rest_v1/media/math/render/svg/0ef36b9a7230888e739f302e1a5d67e36c152de8")

In [461]:
# classics
sade_algo = algorithm(sade(gen=10))
problem_Rast = problem(rastrigin(dim=2)) 
pop = population(problem_Rast, size=pop_size, seed=current_seed)
pop = sade_algo.evolve(pop)
pop.champion_x

array([-0.96373681, -0.03414508])

In [462]:
problem_Rast = problem(rastrigin(dim=2))  # задаем параметры оптимизируемой функции "проблемы" Растиригина

In [463]:
print(problem_Rast)

Problem name: Rastrigin Function
	Global dimension:			2
	Integer dimension:			0
	Fitness dimension:			1
	Number of objectives:			1
	Equality constraints dimension:		0
	Inequality constraints dimension:	0
	Lower bounds: [-5.12, -5.12]
	Upper bounds: [5.12, 5.12]
	Has batch fitness evaluation: false

	Has gradient: true
	User implemented gradient sparsity: false
	Expected gradients: 2
	Has hessians: true
	User implemented hessians sparsity: true
	Expected hessian components: [2]

	Fitness evaluations: 0
	Gradient evaluations: 0
	Hessians evaluations: 0

	Thread safety: basic



In [465]:
pop_rast = population(problem_Rast, size=pop_size, seed=current_seed)  # задаем параметры начальной популяции

pop_rast = few_evols(sade, pop_rast, 10, 3)

In [466]:
pop_rast[0].champion_x  # лучший результат - индивид

array([-0.0070276 ,  0.00469033])

In [467]:
pop_rast[0].champion_f  # лучший результат - значение функции

array([0.01416059])

In [468]:
pop_rast[3]

[(1,
  100,
  0.048530379218153286,
  0.12108423493152899,
  0.0890437854371941,
  1.536547397367902,
  22.736397361776735),
 (3,
  300,
  0.032790263731712344,
  0.2688310985159322,
  0.3379952516851393,
  1.5141863392065895,
  22.752137477263176),
 (5,
  500,
  0.014160587960809323,
  0.26443553902273154,
  0.5506582462092615,
  1.3572876405684007,
  12.115942539472536),
 (7,
  700,
  0.014160587960809323,
  0.26443553902273154,
  0.5506582462092615,
  4.140645024247915,
  11.5330453050835),
 (9,
  900,
  0.014160587960809323,
  0.26443553902273154,
  0.5506582462092615,
  4.140645024247915,
  11.5330453050835)]

In [469]:
pop_rast[4]

,Problem,Method,Pop_Size,Num_Geners,Num_Evoluts,Min_Coordinates,Function_Value
0,Himmelblau,pygmo.core.sade,100,10,10,"[3.0000030217028164, 1.9999978996715322]",[2.858977834867216e-10]
1,Rastrigin Fun,pygmo.core.sade,100,10,3,"[-0.007027601822826161, 0.004690325973629637]",[0.014160587960809323]


### Rosenbrock

**Функция Розенброка**	$f(x) = \displaystyle\sum_{i=1}^{n-1} \big\lbrack 100(x_{i+1} - x_i^2)^2 +(x_i - 1)^2 \big\rbrack$

In [471]:
Image(url= "https://wikimedia.org/api/rest_v1/media/math/render/svg/53f008f82897059e3591d0a9a0aa27b60a1745b4")

In [472]:
problem_Rosen = problem(rosenbrock(dim=2)) # задаем проблему Розенброка

In [475]:
pop_rosen = population(problem_Rosen, size=pop_size, seed=current_seed)
pop_rosen = few_evols(sade, pop_rosen)

In [476]:
pop_rosen[0]

Problem name: Multidimensional Rosenbrock Function
	Global dimension:			2
	Integer dimension:			0
	Fitness dimension:			1
	Number of objectives:			1
	Equality constraints dimension:		0
	Inequality constraints dimension:	0
	Lower bounds: [-5, -5]
	Upper bounds: [10, 10]
	Has batch fitness evaluation: false

	Has gradient: true
	User implemented gradient sparsity: false
	Expected gradients: 2
	Has hessians: false
	User implemented hessians sparsity: false

	Fitness evaluations: 10100
	Gradient evaluations: 0

	Thread safety: constant

Population size: 100

List of individuals: 
#0:
	ID:			1251045422705684049
	Decision vector:	[1.00035, 1.00069]
	Fitness vector:		[1.56896e-07]
#1:
	ID:			13306745122587129611
	Decision vector:	[1.00021, 1.0004]
	Fitness vector:		[5.59468e-08]
#2:
	ID:			2243283290487074856
	Decision vector:	[0.999945, 0.999876]
	Fitness vector:		[2.03062e-08]
#3:
	ID:			14933863474924020957
	Decision vector:	[0.999105, 0.998241]
	Fitness vector:		[8.93731e-07]
#4:
	ID:			1

In [477]:
pop_rosen[0].champion_x

array([1.00005297, 1.00010419])

In [478]:
pop_rosen[0].champion_f

array([3.11168036e-09])

In [479]:
pop_rosen[4]

,Problem,Method,Pop_Size,Num_Geners,Num_Evoluts,Min_Coordinates,Function_Value
0,Himmelblau,pygmo.core.sade,100,10,10,"[3.0000030217028164, 1.9999978996715322]",[2.858977834867216e-10]
1,Rastrigin Fun,pygmo.core.sade,100,10,3,"[-0.007027601822826161, 0.004690325973629637]",[0.014160587960809323]
2,Rosenbrock,pygmo.core.sade,100,10,10,"[1.000052967752323, 1.0001041887458]",[3.1116803553227313e-09]


<a id='gen2'></a>

## Grey Wolf Optimizer (gwo)

### Himmelblau

In [480]:
pop_him = population(problem_Him, size=pop_size, seed=current_seed)  # задаем параметры начальной популяции

pop_him = few_evols(gwo, pop_him)

In [481]:
pop_him[0].champion_x 

array([2.99955458, 2.00506288])

In [482]:
pop_him[0].champion_f

array([0.00039901])

### Rastrigin

In [483]:
pop_rast = population(problem_Rast, size=pop_size, seed=current_seed)

pop_rast = few_evols(gwo, pop_rast, 5, 2)

In [484]:
pop_rast

(Problem name: Rastrigin Function
 	Global dimension:			2
 	Integer dimension:			0
 	Fitness dimension:			1
 	Number of objectives:			1
 	Equality constraints dimension:		0
 	Inequality constraints dimension:	0
 	Lower bounds: [-5.12, -5.12]
 	Upper bounds: [5.12, 5.12]
 	Has batch fitness evaluation: false
 
 	Has gradient: true
 	User implemented gradient sparsity: false
 	Expected gradients: 2
 	Has hessians: true
 	User implemented hessians sparsity: true
 	Expected hessian components: [2]
 
 	Fitness evaluations: 1100
 	Gradient evaluations: 0
 	Hessians evaluations: 0
 
 	Thread safety: basic
 
 Population size: 100
 
 List of individuals: 
 #0:
 	ID:			1251045422705684049
 	Decision vector:	[2.14502e-08, 1.72258e-08]
 	Fitness vector:		[1.49214e-13]
 #1:
 	ID:			13306745122587129611
 	Decision vector:	[2.1583e-08, 2.73215e-09]
 	Fitness vector:		[9.23706e-14]
 #2:
 	ID:			2243283290487074856
 	Decision vector:	[2.12617e-08, 3.29143e-09]
 	Fitness vector:		[9.23706e-14]
 #3:
 	ID

In [485]:
pop_rast[0].champion_x

array([2.15829624e-08, 2.73214609e-09])

In [486]:
pop_rast[0].champion_f

array([9.23705556e-14])

In [487]:
print(problem_Him)

Problem name: <class '__main__.HimmelblauOptimization'>
	Global dimension:			2
	Integer dimension:			0
	Fitness dimension:			1
	Number of objectives:			1
	Equality constraints dimension:		0
	Inequality constraints dimension:	0
	Lower bounds: [-5.12, -5.12]
	Upper bounds: [5.12, 5.12]
	Has batch fitness evaluation: false

	Has gradient: false
	User implemented gradient sparsity: false
	Has hessians: false
	User implemented hessians sparsity: false

	Fitness evaluations: 0

	Thread safety: none



### Rosenbrock

In [488]:
pop_rosen = population(problem_Rosen, size=pop_size, seed=current_seed)

pop_rosen = few_evols(gwo, pop_rosen)

In [489]:
pop_rosen[0].champion_f

array([1.60131646e-05])

In [490]:
pop_rosen[0].champion_x

array([1.00398005, 1.00801746])

In [491]:
pop_rosen[3]

[(1, 6.17926123544877e-05, 6.179058011154061e-05, 6.17899027894503e-05),
 (3, 6.17926123544877e-05, 6.179058011154061e-05, 6.17899027894503e-05),
 (5, 6.17926123544877e-05, 6.179058011154061e-05, 6.17899027894503e-05),
 (7, 6.17926123544877e-05, 6.179058011154061e-05, 6.17899027894503e-05),
 (9, 1.601316463772999e-05, 6.179058011154061e-05, 6.17899027894503e-05)]

In [492]:
pop_rosen[4]

,Problem,Method,Pop_Size,Num_Geners,Num_Evoluts,Min_Coordinates,Function_Value
0,Himmelblau,pygmo.core.sade,100,10,10,"[3.0000030217028164, 1.9999978996715322]",[2.858977834867216e-10]
1,Rastrigin Fun,pygmo.core.sade,100,10,3,"[-0.007027601822826161, 0.004690325973629637]",[0.014160587960809323]
2,Rosenbrock,pygmo.core.sade,100,10,10,"[1.000052967752323, 1.0001041887458]",[3.1116803553227313e-09]
3,Himmelblau,pygmo.core.gwo,100,10,10,"[2.9995545751972155, 2.005062877082653]",[0.0003990115872631824]
4,Rastrigin,pygmo.core.gwo,100,5,2,"[2.1582962408946802e-08, 2.7321460853878126e-09]",[9.237055564881302e-14]
5,Rosenbrock,pygmo.core.gwo,100,10,10,"[1.0039800543656012, 1.0080174624396194]",[1.601316463772999e-05]


<a id='gen3'></a>

## Extended Ant Colony Optimization algorithm (gaco)
сделаю третий, так как интересно как работает, почему бы и нет (:

без вспомогательной функции просто посмотрим на результаты

### Himmelblau

In [511]:
pop_him = population(problem_Him, size=pop_size, seed=current_seed)

pop_him = few_evols(gaco, pop_him, 10, 2)

я бля работаю!!!!


In [512]:
pop_him[0].champion_x

array([2.99999988, 2.00000004])

In [513]:
pop_him[0].champion_f

array([4.87947353e-13])

In [514]:
pop_him[3]

[(1,
  1000,
  0.0019287628954659016,
  13,
  0.08827037278246659,
  0.06403696388562352,
  0.051989068863432374),
 (3,
  1200,
  0.00023316126232297217,
  13,
  0.00023316126232297217,
  0.011307513520496748,
  0.0005815967713786205),
 (5,
  1400,
  1.3542413484468594e-06,
  13,
  2.594980824841773e-05,
  0.0037979395017642137,
  8.429163708649536e-05),
 (7,
  1600,
  2.0968215484494328e-08,
  13,
  3.5749978086843404e-07,
  0.00016852515739418372,
  2.1097055407861529e-07),
 (9,
  1800,
  7.384089993808844e-11,
  13,
  1.6068217309250207e-09,
  3.451292598821709e-06,
  5.569359054689037e-10)]

In [498]:
pop_him[4]

,Problem,Method,Pop_Size,Num_Geners,Num_Evoluts,Min_Coordinates,Function_Value
0,Himmelblau,pygmo.core.sade,100,10,10,"[3.0000030217028164, 1.9999978996715322]",[2.858977834867216e-10]
1,Rastrigin Fun,pygmo.core.sade,100,10,3,"[-0.007027601822826161, 0.004690325973629637]",[0.014160587960809323]
2,Rosenbrock,pygmo.core.sade,100,10,10,"[1.000052967752323, 1.0001041887458]",[3.1116803553227313e-09]
3,Himmelblau,pygmo.core.gwo,100,10,10,"[2.9995545751972155, 2.005062877082653]",[0.0003990115872631824]
4,Rastrigin,pygmo.core.gwo,100,5,2,"[2.1582962408946802e-08, 2.7321460853878126e-09]",[9.237055564881302e-14]
5,Rosenbrock,pygmo.core.gwo,100,10,10,"[1.0039800543656012, 1.0080174624396194]",[1.601316463772999e-05]
6,Himmelblau,pygmo.core.gaco,100,10,2,"[2.9999998776952315, 2.0000000355835263]",[4.879473530775598e-13]


### Rastrigin

In [536]:
pop_rast = population(problem_Rast, size=pop_size, seed=current_seed)

pop_rast = few_evols(gaco, pop_rast, 10, 2)

я бля работаю!!!!


In [537]:
pop_rast[0].champion_f

array([3.85291987e-07])

In [538]:
pop_rast[0].champion_x

array([-4.29992356e-05,  9.65085977e-06])

### Rosenbrock

In [502]:
pop_rosen = population(problem_Rosen, size=pop_size, seed=current_seed)

pop_rosen = few_evols(gaco, pop_rosen, 10, 2)

In [503]:
pop_rosen[0].champion_x

array([0.99952316, 0.99902588])

In [504]:
pop_rosen[0].champion_f

array([2.70071599e-07])

In [539]:
pop_rosen[4]

,Problem,Method,Pop_Size,Num_Geners,Num_Evoluts,Min_Coordinates,Function_Value
0,Himmelblau,pygmo.core.sade,100,10,10,"[3.0000030217028164, 1.9999978996715322]",[2.858977834867216e-10]
1,Rastrigin Fun,pygmo.core.sade,100,10,3,"[-0.007027601822826161, 0.004690325973629637]",[0.014160587960809323]
2,Rosenbrock,pygmo.core.sade,100,10,10,"[1.000052967752323, 1.0001041887458]",[3.1116803553227313e-09]
3,Himmelblau,pygmo.core.gwo,100,10,10,"[2.9995545751972155, 2.005062877082653]",[0.0003990115872631824]
4,Rastrigin,pygmo.core.gwo,100,5,2,"[2.1582962408946802e-08, 2.7321460853878126e-09]",[9.237055564881302e-14]
5,Rosenbrock,pygmo.core.gwo,100,10,10,"[1.0039800543656012, 1.0080174624396194]",[1.601316463772999e-05]
6,Himmelblau,pygmo.core.gaco,100,10,2,"[2.9999998776952315, 2.0000000355835263]",[4.879473530775598e-13]
7,Rastrigin,pygmo.core.gaco,100,10,1,"[0.05023098954377425, -0.05007181550370698]",[0.9897902925446473]
8,Rosenbrock,pygmo.core.gaco,100,10,2,"[0.9995231566008388, 0.9990258785456937]",[2.700715988836341e-07]
9,Rastrigin,pygmo.core.gaco,100,10,1,"[0.05023098954377425, -0.05007181550370698]",[0.9897902925446473]


<a id='gen4m'></a>

## Special Task - Own Defined Optimization Problem